In [18]:
%matplotlib inline 
from nltk.collocations import *
from gensim.models import word2vec
import pandas as pd
from collections import Counter
import functools
import tqdm
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from itertools import product
from __future__ import division
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import nltk, string
from nltk.tokenize import punkt
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pymorphy2
import sys
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import  roc_auc_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import ward, dendrogram
from scipy.cluster import hierarchy
from sklearn.metrics.pairwise import cosine_similarity
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
# exclude = set(punctuation + u'0123456789[]—«»–')
exclude = set(punctuation + u'[]—«»–')
import pylanguagetool
import requests
import json
from gensim.models.keyedvectors import KeyedVectors

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
pd.set_option('display.max_columns', 100)

In [20]:
re1 = re.compile("""
    (?:
        (?:
            (?<!\\d(?:р|г|к))
            (?<!и\\.т\\.(?:д|п))
            (?<!и(?=\\.т\\.(?:д|п)\\.))
            (?<!и\\.т(?=\\.(?:д|п)\\.))
            (?<!руб|коп)
        \\.) |
        [!?\\n]
    )+
    """, re.X)

def safe_split(regex, text):
    res = []
    sear = regex.search(text)
    while sear:
        res.append(text[:sear.end()])
        text = text[sear.end():]
        sear = regex.search(text)
    res.append(text)
    return res

In [21]:
def get_patterns(text):
    tokens = text.split(' ')
    return [[p.tag.POS for p in morph.parse(word)][0] for word in tokens]

In [22]:
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = nltk.stem.porter.PorterStemmer()

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

In [23]:
def normalize_answer(text):
    return ' '.join(re.findall(r"\w+", text)).lower()

def normalize_answer_no_lower(text):
    return ' '.join(re.findall(r"\w+", text))

def sentence_to_word(sentences):
    sentences_in_words = list()
    for sentence in sentences:
        sentences_in_words.append(normalize_answer(sentence).split())
    return sentences_in_words

def text_to_sentence(text):
    sentences = safe_split(re1, text)
    return [s.strip() for s in sentences if s.strip() != '']

def get_max_match_sentance(data_row, w2v = False):
    sentences = text_to_sentence(data_row["paragraph"])
    sentences_in_words = sentence_to_word(sentences)
    question_in_words = sentence_to_word([data_row["question"]])[0]
    overlaps = []
    question_words = set(question_in_words)
    for sentance_id in range(len(sentences_in_words)):
        sentence_words = set(sentences_in_words[sentance_id])
        if w2v:
            overlap = texts_intersection(sentence_words, question_words, model, 0.2)
        else:
            overlap = sentence_words.intersection(question_words)
        overlaps.append(overlap)
    return overlaps

In [24]:
@functools.lru_cache(maxsize=2 ** 19)
def uniq_words(text):
    return set(re.findall("\w+", text))

def calculate_idfs(data):
    counter = Counter()
    uniq_paragraphs = data['paragraph'].unique()
    uniq_questions = data['question'].unique()
    for paragraph in tqdm.tqdm(uniq_paragraphs, desc="calc idf for paragraph"):
        set_words = uniq_words(paragraph)
        counter.update(set_words)
    for question in tqdm.tqdm(uniq_questions, desc="calc idf for question"):
        set_words = uniq_words(question)
        counter.update(set_words)
    num_docs = uniq_paragraphs.shape[0] + uniq_questions.shape[0]
    idfs = {}
    for word in counter:
        idfs[word] = np.log(num_docs / counter[word])
    return idfs

In [25]:
def preprocess(text):
    
    buf = ''.join(ch for ch in text if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(buf.lower())
    lemmatizer = MorphAnalyzer()
    lemmas = []
    for t in tokens[:]:
        if not t in stopwords.words('russian'):
            try:
                lemma = lemmatizer.parse(t)[0].normal_form
            except: 
                lemma = t
            lemmas.append(lemma)
    return ' '.join(lemmas)

In [26]:
def texts_intersection(text1, text2, word2vec, threshold):
    words1 = set(text1)
    words2 = set(text2)
    result = []
    for word1 in words1:
        for word2 in words2:
            if word1 == word2:
                similarity = 1.0
            elif word1 in word2vec and word2 in word2vec:
                similarity = word2vec.similarity(word1, word2)
            else:
                similarity = 0.0
            if similarity >= threshold:
                result.append(word1)
                result.append(word2)
    return ' '.join(result)

In [27]:
patterns = ['ADJF+NOUN','PRTS+NOUN','NOUN+NOUN', 'NOUN+VERB', 'VERB+ADJF', 'VERB+NOUN']
morph = pymorphy2.MorphAnalyzer()
def match(ngram, patterns = patterns):
    index = []
    for word in ngram:
        buf = [(p.normal_form, p.tag.POS) for p in morph.parse(word)]
        index.append((word,buf))
    pos_tagging = product(*[ind[1] for ind in index])
    possible_patterns = map(lambda pos_tag: list(zip(*pos_tag)), pos_tagging)
    possible_patterns = map(lambda pattern: [pattern[0], map(lambda grammeme: grammeme, pattern[1])], possible_patterns)
    possible_patterns = map(lambda pattern: (pattern[0], '+'.join(pattern[1])), possible_patterns)
    for pattern in possible_patterns:
        if pattern[1] in patterns:
            return pattern
    return None        

def get_pattern(text):
    tokens = [x for x in normalize(text)]
    pattern_coll2  = nltk.FreqDist() 
    coll2 = nltk.FreqDist(nltk.bigrams(tokens))
    for c2 in coll2:
        try:
            p = match(c2)
            if p != None:
                collocation = ' '.join(p[0])
                if collocation in pattern_coll2:
                    pattern_coll2[collocation] += coll2[c2]
                else:
                    pattern_coll2[collocation] = coll2[c2]
        except:
            pass
    return pattern_coll2

## Data

In [28]:
dftrain, dftest = pd.read_csv("data/train_task1_latest.csv"), pd.read_csv("data/sdsj_A_test.csv")

In [12]:
# # add pattern generated questions
# dftrain_generated = pd.read_csv("../data/taskA_generated_questions_p.csv", sep=';')
# question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
# for row in dftrain_generated.iterrows():
#     dftrain_generated.loc[row[0], 'question_id'] = question_id
#     question_id += 1
# dftrain_generated['target'] = 0.0
# dftrain_generated['generated_p'] = 1.0
# dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p']
# dftrain['generated_p'] = 0.0
# dftrain = dftrain.append(dftrain_generated, ignore_index=True)
# dftrain.fillna(0, inplace = True)

In [13]:
# # add pattern_2 generated questions
# dftrain_generated = pd.read_csv("../data/taskA_generated_questions_p_2.csv", sep=';')
# dftrain_generated = dftrain_generated.sample(10000)
# question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
# for row in dftrain_generated.iterrows():
#     dftrain_generated.loc[row[0], 'question_id'] = question_id
#     question_id += 1
# dftrain_generated['target'] = 0.0
# dftrain_generated['generated_p_2'] = 1.0
# dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_2']
# dftrain['generated_p_2'] = 0.0
# dftrain = dftrain.append(dftrain_generated, ignore_index=True)
# dftrain.fillna(0, inplace = True)

In [14]:
# add pattern_3 generated questions
dftrain_generated = pd.read_csv("../data/taskA_generated_questions_p_3.csv", sep=';')
dftrain_generated = dftrain_generated.sample(10000)
question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
for row in dftrain_generated.iterrows():
    dftrain_generated.loc[row[0], 'question_id'] = question_id
    question_id += 1
dftrain_generated['target'] = 0.0
dftrain_generated['generated_p_3'] = 1.0
dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_3']
dftrain['generated_p_3'] = 0.0
dftrain = dftrain.append(dftrain_generated, ignore_index=True)
dftrain.fillna(0, inplace = True)

In [ ]:
# add pattern_4 generated questions
dftrain_generated = pd.read_csv("../data/taskA_generated_questions_p_4.csv", sep=';')
dftrain_generated = dftrain_generated.sample(10000)
question_id = np.max([dftrain.question_id.max(), dftest.question_id.max()]) + 1
for row in dftrain_generated.iterrows():
    dftrain_generated.loc[row[0], 'question_id'] = question_id
    question_id += 1
dftrain_generated['target'] = 0.0
dftrain_generated['generated_p_4'] = 1.0
dftrain_generated.columns = ['paragraph_id', 'paragraph', 'question', 'question_id', 'target', 'generated_p_4']
dftrain['generated_p_4'] = 0.0
dftrain = dftrain.append(dftrain_generated, ignore_index=True)
dftrain.fillna(0, inplace = True)

## Baseline

In [15]:
idfs = calculate_idfs(dftrain.append(dftest, ignore_index = True))

calc idf for question: 100%|██████████| 120483/120483 [00:04<00:00, 29182.71it/s]


In [16]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        question = uniq_words(row.question)
        paragraph = uniq_words(row.paragraph)
        df.loc[index, 'len_paragraph'] = len(paragraph)
        df.loc[index, 'len_question'] = len(question)
        df.loc[index, 'len_intersection'] = len(paragraph & question)
        df.loc[index, 'idf_question'] = np.sum([idfs.get(word, 0.0) for word in question])
        df.loc[index, 'idf_paragraph'] = np.sum([idfs.get(word, 0.0) for word in paragraph])
        df.loc[index, 'idf_intersection'] = np.sum([idfs.get(word, 0.0) for word in paragraph & question])
    df['relative_question_len'] = df['len_question'] / df['len_paragraph']
    df['relative_intersection_len'] = df['len_intersection'] / df['len_paragraph']
    df['relative_intersection_question_len'] = df['len_intersection'] / df['len_question']
    df['relative_question_idf'] = df['idf_question'] / df['idf_paragraph']
    df['relative_intersection_idf'] = df['idf_intersection'] / df['idf_paragraph']
    df['relative_intersection_question_idf'] = df['idf_intersection'] / df['idf_question']
    df['word_idf_paragraph'] = df['idf_paragraph'] / df['len_paragraph']
    df['word_idf_question'] = df['idf_question'] / df['len_question']
    df['word_idf_intersection'] = df['idf_intersection'] / df['len_intersection'] 
    df.fillna(0, inplace = True)

build features for test: 100%|██████████| 74286/74286 [05:06<00:00, 242.07it/s]


## Data mining


**lemmarize**

In [17]:
%%time
# train - paragraph
df = pd.DataFrame(dftrain.paragraph.unique())
df.columns = ['paragraph']
df['paragraph_lemmatized'] = df['paragraph'].map(lambda x: preprocess(x))
dftrain = dftrain.merge(df, on = 'paragraph', how = 'left')
print('dftrain complete - paragraph_lemmatized')

# train - question
df = pd.DataFrame(dftrain.question.unique())
df.columns = ['question']
df['question_lemmatized'] = df['question'].map(lambda x: preprocess(x))
dftrain = dftrain.merge(df, on = 'question', how = 'left')
print('dftrain complete - question_lemmatized')

# test - paragraph
df = pd.DataFrame(dftest.paragraph.unique())
df.columns = ['paragraph']
df['paragraph_lemmatized'] = df['paragraph'].map(lambda x: preprocess(x))
dftest = dftest.merge(df, on = 'paragraph', how = 'left')
print('dftest complete - paragraph_lemmatized')

# test - question
df = pd.DataFrame(dftest.question.unique())
df.columns = ['question']
df['question_lemmatized'] = df['question'].map(lambda x: preprocess(x))
dftest = dftest.merge(df, on = 'question', how = 'left')
print('dftest complete - question_lemmatized')

dftrain complete - paragraph_lemmatized
dftrain complete - question_lemmatized
dftest complete - paragraph_lemmatized
dftest complete - question_lemmatized
CPU times: user 2h 45min 55s, sys: 8min 10s, total: 2h 54min 5s
Wall time: 2h 57min 29s


**word2vec similarity**

In [18]:
# load model

w2v = KeyedVectors.load_word2vec_format("./data/ruwikiruscorpora_0_300_20.bin.gz", binary=True)
new_vocab = {}
[new_vocab.update({k[0].replace("::", "_").split('_')[0]: k[1]}) for k in w2v.vocab.items()]
w2v.vocab = new_vocab

# or build model

# model = word2vec.Word2Vec(np.append(dftrain.paragraph_lemmatized, 
#                                     np.append(dftrain[dftrain.generated_p_2 ==0][dftrain.generated_p ==0].question_lemmatized, 
#                                               dftest.paragraph_lemmatized)),
#                  size=100, 
#                  window=5, 
#                  min_count=5, 
#                  workers=4)
# fname = '../results/word2vec_model'
# model.save(fname)
# # model = word2vec.Word2Vec.load(fname)

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word2vec"):  
        question = row['question_lemmatized']
        paragraph = row['paragraph_lemmatized']
        w2v_similarities = []
        for question_word in question.split(' '):
            max_similarity = 0.0
            for paragraph_word in paragraph.split(' '):
                if (question_word in model.wv.vocab) and (paragraph_word in model.wv.vocab):
                    similarity = model.similarity(question_word, paragraph_word)
                    if similarity > max_similarity:
                        max_similarity = similarity
            w2v_similarities.append(max_similarity)
        w2v_similarities = [x for x in w2v_similarities if x > 0.2]
        
        if len(w2v_similarities) == 0:
            df.loc[index, 'word2vec_similarity_max'] = 0
            df.loc[index, 'word2vec_similarity_min'] = 0
        else:
            df.loc[index, 'word2vec_similarity_max'] = np.max(w2v_similarities)
            df.loc[index, 'word2vec_similarity_min'] = np.min(w2v_similarities)
        df.loc[index, 'word2vec_similarity_mean'] = np.mean(w2v_similarities)
        df.loc[index, 'word2vec_similarity_std'] = np.std(w2v_similarities)
    df.fillna(0, inplace = True)

calculating word2vec:  34%|███▍      | 54860/159171 [03:17<06:41, 259.76it/s]

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        intersection = texts_intersection(row.paragraph_lemmatized, row.question_lemmatized, model, 0.2)
        df.loc[index, 'len_intersection_w2v'] = len(intersection)
        df.loc[index, 'idf_intersection_w2v'] = np.sum([idfs.get(word, 0.0) for word in intersection])
    df['relative_intersection_len_w2v'] = df['len_intersection_w2v'] / df['len_paragraph']
    df['relative_intersection_question_len_w2v'] = df['len_intersection_w2v'] / df['len_question']
    df['relative_intersection_idf_w2v'] = df['idf_intersection_w2v'] / df['idf_paragraph']
    df['relative_intersection_question_idf_w2v'] = df['idf_intersection_w2v'] / df['idf_question']
    df['word_idf_intersection_w2v'] = df['idf_intersection_w2v'] / df['len_intersection'] 
    df.fillna(0, inplace = True)

**word mover distance**

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        df.loc[index, 'word_mover_distance'] = model.wmdistance(row['paragraph'], row['question'])
        df.loc[index, 'word_mover_distance_lemm'] = model.wmdistance(row['paragraph_lemmatized'], row['question_lemmatized'])
        wmds = []
        for paragraph_sentence in text_to_sentence(row['paragraph']):
            wmds.append(model.wmdistance(paragraph_sentence, row['question']))
        df.loc[index, 'word_mover_sentence_distance_mean'] = np.mean(wmds)
        df.loc[index, 'word_mover_sentence_distance_std'] = np.std(wmds)
        df.loc[index, 'word_mover_sentence_distance_max'] = np.max(wmds)
        df.loc[index, 'word_mover_sentence_distance_min'] = np.min(wmds)

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating word mover distance for " + name):
        question = row.question
        question_lemm = row.question_lemmatized
        question_len = len(question_lemm)
        similarities = []
        similarities_lemm = []
        if int(question_len / 3) != 0:
            for i in range(0, question_len, int(question_len / 3)):
                question_chunk = question[i:i + int(question_len / 3)]
                similarities.append(model.wmdistance(question_chunk, row['paragraph']))
                question_chunk_lemm = question_lemm[i:i + int(question_len / 3)]
                similarities_lemm.append(model.wmdistance(question_chunk_lemm, row['paragraph_lemmatized']))
        elif int(question_len / 2) != 0:
            for i in range(0, question_len, int(question_len / 2)):
                question_chunk = question[i:i + int(question_len / 2)]
                similarities.append(model.wmdistance(question_chunk, row['paragraph']))
                question_chunk_lemm = question_lemm[i:i + int(question_len / 2)]
                similarities_lemm.append(model.wmdistance(question_chunk_lemm, row['paragraph_lemmatized']))
        else:
            question_chunk = question
            similarities.append(model.wmdistance(question_chunk, row['paragraph']))
            question_chunk_lemm = question_lemm
            similarities_lemm.append(model.wmdistance(question_chunk_lemm, row['paragraph_lemmatized']))
        df.loc[index, 'word_mover_sentence_distance_split_max'] = np.max(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_min'] = np.min(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_mean'] = np.mean(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_std'] = np.std(similarities)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_max'] = np.max(similarities_lemm)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_min'] = np.min(similarities_lemm)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_mean'] = np.mean(similarities_lemm)
        df.loc[index, 'word_mover_sentence_distance_split_lemm_std'] = np.std(similarities_lemm)

**max match sentence**

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating max overlap"):        
        overlaps = get_max_match_sentance(row)
        
        # vars
        a = [len(x) / len(row['paragraph']) for x in overlaps]
        b = [len(x) / len(row['question']) for x in overlaps]
        c = [np.sum([idfs.get(word, 0.0) for word in overlap]) for overlap in overlaps]
        
        # max
        max_overlap = ''
        max_overlap_idf = 0
        for overlap in overlaps:
            overlap_idf = np.sum([idfs.get(word, 0.0) for word in overlap])
            if overlap_idf > max_overlap_idf:
                max_overlap_idf = overlap_idf
                max_overlap = overlap
        df.loc[index, 'answer'] = max_overlap
        
        df.loc[index, 'len_overlap_relative_paragraph_max'] = np.max(a)
        df.loc[index, 'len_overlap_relative_question_max'] = np.max(b)
        df.loc[index, 'idf_overlap_max'] = np.max(c)
        
        
        
        # means
        df.loc[index, 'len_overlap_relative_paragraph_mean'] = np.mean(a)
        df.loc[index, 'len_overlap_relative_question_mean'] = np.mean(b)
        df.loc[index, 'idf_overlap_mean'] = np.mean(c)
        
        # stds
        df.loc[index, 'len_overlap_relative_paragraph_std'] = np.std(a)
        df.loc[index, 'len_overlap_relative_question_std'] = np.std(b)
        df.loc[index, 'idf_overlap_std'] = np.std(c)


In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating max overlap"):        
        overlaps = get_max_match_sentance(row, w2v=True)
        
        # vars
        a = [len(x) / len(row['paragraph']) for x in overlaps]
        b = [len(x) / len(row['question']) for x in overlaps]
        c = [np.sum([idfs.get(word, 0.0) for word in overlap]) for overlap in overlaps]
        
        # max
        df.loc[index, 'len_overlap_relative_paragraph_max_w2v'] = np.max(a)
        df.loc[index, 'len_overlap_relative_question_max_w2v'] = np.max(b)
        df.loc[index, 'idf_overlap_max_w2v'] = np.max(c)
        
        # max
        max_overlap = ''
        max_overlap_idf = 0
        for overlap in overlaps:
            overlap_idf = np.sum([idfs.get(word, 0.0) for word in overlap])
            if overlap_idf > max_overlap_idf:
                max_overlap_idf = overlap_idf
                max_overlap = overlap
        df.loc[index, 'answer_w2v'] = max_overlap
        
        # means
        df.loc[index, 'len_overlap_relative_paragraph_mean_w2v'] = np.mean(a)
        df.loc[index, 'len_overlap_relative_question_mean_w2v'] = np.mean(b)
        df.loc[index, 'idf_overlap_mean_w2v'] = np.mean(c)
        
        # stds
        df.loc[index, 'len_overlap_relative_paragraph_std_w2v'] = np.std(a)
        df.loc[index, 'len_overlap_relative_question_std_w2v'] = np.std(b)
        df.loc[index, 'idf_overlap_std_w2v'] = np.std(c)
        
        


**metric learning**

https://all-umass.github.io/metric-learn/

In [ ]:
w2v -> covariance matrix?

In [12]:
from metric_learn import Covariance
from metric_learn import LMNN

In [62]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
vectorizer.fit(set(np.append(pd.unique([x for x in dftrain.paragraph.values]),
                                     np.append(pd.unique([x for x in dftrain.question.values]),
                                              pd.unique([x for x in dftest.paragraph.values])))))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [76]:
vectorizer.transform(dftrain.paragraph.values[:25])

<1x2022270 sparse matrix of type '<class 'numpy.float64'>'
	with 262 stored elements in Compressed Sparse Row format>

In [66]:
cov  = Covariance().fit(np.matrix(vectorizer.transform(dftrain.paragraph.values[:25])))

AttributeError: 'float' object has no attribute 'shape'

In [7]:
cov.metric(dftrain.paragraph.values[0], dftrain.question.values[0])

NameError: name 'cov' is not defined

In [ ]:
cov.metric('В годовом отчете 1998 г. МВФ впервые заявил, что эффективный банковский надзор должен быть непрерывным.. это может быть обеспечено, главным образом, за счет документарного надзора как на микропруденциальном, так и на макропруденциальном уровне… макропруденциальный анализ основан на исследовании рынка и макроэкономической информации, в фокусе его внимания находятся ключевые рынки активов, финансовые посредники, макроэкономическое развитие и потенциальные дисбалансы . Для макропруденциального анализа в 2000 г. МВФ разработал макропруденциальные индикаторы (macroprudential indicators), которые на следующий год были переименованы в показатели финансовой устойчивости (financial soundness indicators). В 2003 г. программа расчета макропоказателей финансовой устойчивости была внедрена в Банке России.', 'Сколько всего в устойчивости financial soundness indicators?	')

In [ ]:
cov.transform(dftrain.question.values[0])

**similarity**

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
vectorizer.fit(set(np.append(pd.unique([x for x in dftrain.paragraph_lemmatized.values]),
                                     np.append(pd.unique([x for x in dftrain[dftrain.generated_p_2 ==0][dftrain.generated_p ==0].question_lemmatized.values]),
                                              pd.unique([x for x in dftest.paragraph_lemmatized.values])))))

In [ ]:
# calculate cummulative nlpS
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating nlps for " + name):
        vectors = vectorizer.transform([row.paragraph_lemmatized, row.question_lemmatized])
        df.loc[index, 'similarity_tfid'] = (vectors * vectors.T)[0, 1]

In [ ]:
# calculate cummulative nlpS - split question to chunks
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="calculating nlps for " + name):
        question = row.question_lemmatized
        question_str = question
        question_len = len(question_str)
        similarities = []
        if int(question_len / 3) != 0:
            for i in range(0, question_len, int(question_len / 3)):
                question_chunk = question[i:i + int(question_len / 3)]
                vectors = vectorizer.transform([row.paragraph_lemmatized, question_chunk])
                similarities.append((vectors * vectors.T)[0, 1])
        elif int(question_len / 2) != 0:
            for i in range(0, question_len, int(question_len / 2)):
                question_chunk = question[i:i + int(question_len / 2)]
                vectors = vectorizer.transform([row.paragraph_lemmatized, question_chunk])
                similarities.append((vectors * vectors.T)[0, 1])
        else:
            question_chunk = question
            vectors = vectorizer.transform([row.paragraph_lemmatized, question_chunk])
            similarities.append((vectors * vectors.T)[0, 1])
        df.loc[index, 'similarity_tfid_split_max'] = np.max(similarities)
        df.loc[index, 'similarity_tfid_split_min'] = np.min(similarities)
        df.loc[index, 'similarity_tfid_split_mean'] = np.mean(similarities)
        df.loc[index, 'similarity_tfid_split_std'] = np.std(similarities)

**словосочетания**

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        question_patterns = list(get_pattern(row.question).keys())
        paragraph_patterns = list(get_pattern(row.paragraph).keys())
        vectorizer = TfidfVectorizer(ngram_range=(1,3))
        vectorizer.fit(np.append(question_patterns, paragraph_patterns))
        similarity_pattern = []
        for pattern in question_patterns:
            vectors = vectorizer.transform(np.append(pattern, paragraph_patterns))
            # eliminate diagonal 1'ses
            matrix = (vectors * vectors.T)
            n = matrix.shape[0]
            matrix[range(n), range(n)] = 0
            similarity_pattern.append(matrix[0].max())
        # get max similarity
        df.loc[index, 'similarity_pattern_mean'] = np.mean(similarity_pattern)
        df.loc[index, 'similarity_pattern_std'] = np.std(similarity_pattern)
        if len(similarity_pattern) == 0:
            df.loc[index, 'similarity_pattern_max'] = 0
            df.loc[index, 'similarity_pattern_min'] = 0
        else:
            df.loc[index, 'similarity_pattern_max'] = np.max(similarity_pattern)
            df.loc[index, 'similarity_pattern_min'] = np.min(similarity_pattern)
    df.fillna(0, inplace = True)

### Unique words

In [ ]:
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="grammar for " + name):
        question_lemmatized = row.question_lemmatized
        
        unique_words_len = len(set(question_lemmatized))
        df.loc[index, 'unique_words_relative_to_question'] = unique_words_len / row.len_question
        
        word_occurences = []
        for word in question_lemmatized.split(' '):
            word_occurences.append(question_lemmatized.count(word))
            
        df.loc[index, 'word_ocurences_mean'] = np.mean(word_occurences)
        df.loc[index, 'word_ocurences_std'] = np.std(word_occurences)
        df.loc[index, 'word_ocurences_max'] = np.max(word_occurences)
        df.loc[index, 'word_ocurences_max_relative_to_question'] = np.max(word_occurences) / row.len_question
        

**language tool**

In [ ]:
# add multiproccessing
# https://stackoverflow.com/questions/34512646/how-to-speed-up-api-requests
# https://stackoverflow.com/questions/33550312/multiprocessing-writing-to-pandas-dataframe

In [39]:
df = pd.DataFrame()
df['question'] = np.append(dftrain.question.unique(), dftest.question.unique())
for index, row in tqdm.tqdm(df.iterrows(), total = df.shape[0]):
    matches = requests.get('http://localhost:8081/v2/check', params={'text':row.question, 'language':'ru-RU'}).json().get('matches')
    matches_dict = {}
    for match in matches:
        issue_type = match.get('rule').get('issueType')
        issue_count = matches_dict.get(issue_type)
        
        if issue_count is None:
            matches_dict.update({issue_type: 1})
        else:
            matches_dict.update({issue_type: issue_count + 1})
            
    
    for item in matches_dict.items():
        df.loc[index, item[0]] = item[1]
df.fillna(0, inplace = True)

dftrain = dftrain.merge(df, on = 'question', how = 'left')
dftest = dftest.merge(df, on = 'question', how = 'left')


  0%|                                                                             | 5/82208 [00:05<24:03:41,  1.05s/it]


KeyboardInterrupt: 

### Добавим проверку грамматики

In [ ]:
'\', \''.join(dftrain.columns)

In [84]:
# columns = ['len_paragraph', 'len_question', 'len_intersection', 'idf_question', 'idf_paragraph', 'idf_intersection', 'relative_question_len', 'relative_intersection_len', 'relative_intersection_question_len', 'relative_question_idf', 'relative_intersection_idf', 'relative_intersection_question_idf', 'word_idf_paragraph', 'word_idf_question', 'word_idf_intersection', 'word2vec_similarity_max', 'word2vec_similarity_min', 'word2vec_similarity_mean', 'word2vec_similarity_std', 'len_intersection_w2v', 'idf_intersection_w2v', 'relative_intersection_len_w2v', 'relative_intersection_question_len_w2v', 'relative_intersection_idf_w2v', 'relative_intersection_question_idf_w2v', 'word_idf_intersection_w2v', 'word_mover_distance', 'word_mover_distance_lemm', 'word_mover_sentence_distance_mean', 'word_mover_sentence_distance_std', 'word_mover_sentence_distance_max', 'word_mover_sentence_distance_min', 'word_mover_sentence_distance_split_max', 'word_mover_sentence_distance_split_min', 'word_mover_sentence_distance_split_mean', 'word_mover_sentence_distance_split_std', 'word_mover_sentence_distance_split_lemm_max', 'word_mover_sentence_distance_split_lemm_min', 'word_mover_sentence_distance_split_lemm_mean', 'word_mover_sentence_distance_split_lemm_std', 'len_overlap_relative_paragraph_max', 'len_overlap_relative_question_max', 'idf_overlap_max', 'len_overlap_relative_paragraph_mean', 'len_overlap_relative_question_mean', 'idf_overlap_mean', 'len_overlap_relative_paragraph_std', 'len_overlap_relative_question_std', 'idf_overlap_std', 'len_overlap_relative_paragraph_max_w2v', 'len_overlap_relative_question_max_w2v', 'idf_overlap_max_w2v', 'len_overlap_relative_paragraph_mean_w2v', 'len_overlap_relative_question_mean_w2v', 'idf_overlap_mean_w2v', 'len_overlap_relative_paragraph_std_w2v', 'len_overlap_relative_question_std_w2v', 'idf_overlap_std_w2v', 'similarity_tfid', 'similarity_tfid_split_max', 'similarity_tfid_split_min', 'similarity_tfid_split_mean', 'similarity_tfid_split_std', 'similarity_pattern_mean', 'similarity_pattern_std', 'similarity_pattern_max', 'similarity_pattern_min', 'unique_words_relative_to_question', 'word_ocurences_mean', 'word_ocurences_std', 'word_ocurences_max', 'word_ocurences_max_relative_to_question']

# dftrainfull = xgb.DMatrix(dftrain[dftrain.generated_p == 0][dftrain.generated_p_2 == 0][columns], dftrain[dftrain.generated_p == 0][dftrain.generated_p_2 == 0].generated_p_3)
# # dftrainfull = xgb.DMatrix(dftrain[columns], dftrain[['generated_p', 'generated_p_2', 'generated_p_3']].sum(axis=1))
# params = {}
# params['objective'] = 'binary:logistic'


# # triple xgb
# a = []
# b = []
# for i in range(0, 3):
#     params_i = params
#     params_i['colsample_bylevel'] = 0.5 + i * 0.25
#     params_i['colsample_bytree'] = 0.5 + i * 0.25
#     bst = xgb.train(params, dtrainfull, num_boost_round=100)
#     a.append(bst.predict(xgb.DMatrix(dftrain[columns])))
#     b.append(bst.predict(xgb.DMatrix(dftest[columns])))

# dftrain['grammar_prediction'] = [sum(e)/len(e) for e in zip(*a)]
# dftest['grammar_prediction'] = [sum(e)/len(e) for e in zip(*b)]

# score = roc_auc_score(dftrain.generated_p_3, dftrain['grammar_prediction'])
# print('e - roc_auc_score: ' + str(score))

In [87]:
# pd.options.display.max_colwidth = 10000
# dftest[['question', 'grammar_prediction']]#[dftest.grammar_prediction < 0.5]

,question,grammar_prediction
0,В какой мифологии два ворона Хугин и Мунин шептали новости в уши бога Одина?,0.001077
1,"Когда инсайдер покупает или гипотезы чего эволюционно устойчивых стратегий развития стратегий войн, развития стратегий войн, развития стратегий?",0.881770
2,"Какие предположения, по мнению Ньютона, допустимы в натуральной философии (то есть физике)?",0.000781
3,В каких условиях метаболизм поддерживает порядок за счёт создания беспорядка?,0.000652
4,"Каким мелодиям Прокофьева особенно не укладывается в таблицы именно этих языков, алфавит которых укладывается в места, недоступные для мощности в существенной мере способствовал систематизации его чувства, опыт?",0.878052
5,Какая песня стала гимном панк-движения?,0.002892
6,Каким методом модель ЦП доводится до ума?,0.002890
7,При какой рекомбинации у высших грибов диплоидное (зиготическое) ядро без периода покоя делится редукционно с образованием тетрады?,0.001726
8,"Когда деяние совершается в соответствующую уголовно правовую оценку общественной опасностью и законодатель сочтёт нужным создать соответствующую уголовно правовую норму заработной платы и заражения вирусами вызываются такие заболевания, как настоящий щит?",0.952995
9,Где может исполняться программа на языке третьего поколения?,0.001266


## Обработка типа вопроса

In [4]:
question_types_keys = ['кто', 'кого', 'когда', 'почему', 'зачем', 'отчего', 'куда', 'кому', 'чему', 'как', 'о ком', 'чем',
                  'на чем', 'как', 'сколько', 'где', 'какого', 'откуда', 'чего', 'кем', 'чем', 'чей', 'который', 'какой',
                  'какая', 'какие', 'какое', 'каких', 'какие', 'чья', 'чьи', 'чье']
def update_question_types_dict(question, answer_patterns):
    
    '''
    question - text
    answer_patterns - patterns of question (intersection) paragraph
    
    function remebers what patterns correlate with question type
    '''
    
    
    for word in question.split(' '):
        if word in question_types_keys:
            
            patterns_dict = question_types_dict.get(word)
            
            if patterns_dict is None:
                patterns_dict = {}
            
            for answer_pattern in answer_patterns:
                answer_pattern_count = patterns_dict.get(answer_pattern)
                
                if answer_pattern_count is None:
                    answer_pattern_count = 0
                    
                patterns_dict.update({answer_pattern:answer_pattern_count+1})
            
            question_types_dict.update({word:patterns_dict})
            
            break

In [31]:
# заполним словарь
question_types_dict = {}
for group_name, group in dftrain[dftrain.target == 1].groupby(['question', 'answer']):
    update_question_types_dict(group_name[0], group_name[1])


 45 калибр , 450 калибр где чаще употребляется это сочетание?


In [ ]:
# рассчитаем, насколько ответ для вопроса схож со средними значениями 
for name, df in [('train', dftrain), ('test', dftest)]:
    for index, row in tqdm.tqdm(df.groupby(['question', 'answer']), desc="q type for " + name):
        question = group_name[0]
        answer = group_name[1]
        
        

In [ ]:
question_types_dict = {}

## Сохраним данные

In [88]:
dftrain.to_csv('../data/train_task1_latest_pred.csv', sep=';', index= False)
dftest.to_csv('../data/test_task1_latest_pred.csv', sep=';', index= False)

## Загрузим даные

In [ ]:
# dftrain = pd.read_csv('../data/train_task1_latest_pred.csv', sep=';')
# dftest = pd.read_csv('../data/test_task1_latest_pred.csv', sep=';')

### Let us see

In [ ]:
pd.options.display.max_colwidth = 10000

### Patterns